# Chapter 14
## Deep Computer Vision Using Convolutional Neural Networks

In [1]:
# Utilities to load popular datasets and artificial data generators.
from sklearn.datasets import load_sample_images 

In [3]:
import tensorflow as tf

# Testen, ob TensorFlow korrekt installiert wurde
print("TensorFlow version:", tf.__version__)
print("Available CPUs:", tf.config.list_physical_devices('CPU'))

TensorFlow-Version: 2.18.0
Verfügbare CPUs: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [8]:
images = load_sample_images()["images"]
# This layers crops the central portion of the images to a target size.
images = tf.keras.layers.CenterCrop(height=70, width=120)(images)
# A preprocessing layer which rescales input values to a new range.
images = tf.keras.layers.Rescaling(scale=1 / 255)(images)

In [9]:
# two sample images, height, width, RGB
images.shape

TensorShape([2, 70, 120, 3])

In [17]:
# Convolutional2D layer (32 random filters, 7x7 filter size)
# Padding "same": Shape is conservated
# Padding "valid": Filter applied only on "valid" data (no padding, adding on boarders) 
# Strides: kernel jumps in pixel
conv_layer = tf.keras.layers.Conv2D(filters=32, kernel_size=7, padding="same", strides=1)
fmaps = conv_layer(images)
fmaps.shapekerne

TensorShape([2, 70, 120, 32])

In [20]:
kernels, biases = conv_layer.get_weights()
# [kernel_height,kernel_width,input_channels,output_channels]
print(kernels.shape)
# [output_channels]
print(biases.shape)


(7, 7, 3, 32)
(32,)


### Hyperparameters
- filters
- kernel_size
- padding
- strides
- activation
- kernel_initializer